In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import cv2
import os
from time import time

# Configuration
os.makedirs("results", exist_ok=True)
SEQ_LEN = 5
IMG_SIZE = (64, 64)
BATCH_SIZE = 4

# Construction du modèle
def build_model():
    inputs = Input(shape=(SEQ_LEN, *IMG_SIZE, 1))
    
    # Encoder
    x = ConvLSTM2D(16, (3, 3), padding='same', return_sequences=True)(inputs)
    x = BatchNormalization()(x)
    
    # Decoder
    x = TimeDistributed(Conv2D(8, (3, 3), padding='same', activation='relu'))(x)
    outputs = TimeDistributed(Conv2D(1, (3, 3), padding='same', activation='sigmoid'))(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')
    return model

# Pré-traitement des données
def preprocess_data(path, sample_size=100):
    data = np.load(path, mmap_mode='r')[:sample_size]
    processed = []
    for frame in data:
        frame = cv2.resize(frame.astype('float32'), IMG_SIZE) / 255.0
        processed.append(frame[..., np.newaxis])
    return np.array(processed)

def create_sequences(frames):
    sequences = []
    for i in range(len(frames) - SEQ_LEN + 1):
        sequences.append(frames[i:i+SEQ_LEN])
    return np.array(sequences)

# Entraînement
def train_model(model, train_data):
    sequences = create_sequences(train_data)
    history = model.fit(
        sequences, sequences,
        validation_split=0.2,
        epochs=10,
        batch_size=BATCH_SIZE,
        callbacks=[EarlyStopping(patience=2)],
        verbose=1
    )
    return history, sequences

# Détection des anomalies
def detect_anomalies(model, data, train_sequences):
    # Calcul du seuil
    train_recon = model.predict(train_sequences, batch_size=BATCH_SIZE)
    train_errors = np.mean(np.square(train_sequences - train_recon), axis=(1,2,3,4))
    threshold = np.percentile(train_errors, 95)
    
    # Évaluation sur les données de test
    test_sequences = create_sequences(data)
    test_recon = model.predict(test_sequences, batch_size=BATCH_SIZE)
    test_errors = np.mean(np.square(test_sequences - test_recon), axis=(1,2,3,4))
    
    anomalies = test_errors > threshold
    return {
        'threshold': threshold,
        'anomaly_percentage': np.mean(anomalies) * 100,
        'errors': test_errors,
        'anomalies': anomalies
    }

# Visualisation
def visualize_results(history, results, test_data):
    # Courbe d'apprentissage
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title('Courbe d\'apprentissage')
    plt.legend()
    
    # Distribution des erreurs
    plt.subplot(1, 2, 2)
    plt.hist(results['errors'], bins=50, alpha=0.7)
    plt.axvline(results['threshold'], color='r', linestyle='--')
    plt.title(f"Anomalies: {results['anomaly_percentage']:.2f}%")
    plt.savefig("results/analysis.png")
    plt.close()
    
    # Sauvegarde des résultats
    with open("results/report.txt", "w") as f:
        f.write(f"Pourcentage d'anomalies: {results['anomaly_percentage']:.2f}%\n")
        f.write(f"Seuil utilisé: {results['threshold']:.4f}\n")

# Pipeline principal
def main():
    try:
        # Initialisation
        print("1. Construction du modèle...")
        model = build_model()
        model.summary()
        
        # Entraînement
        print("\n2. Chargement des données d'entraînement...")
        train_data = preprocess_data("UCSDped1_train.npy")
        history, train_sequences = train_model(model, train_data)
        
        # Détection
        print("\n3. Détection des anomalies...")
        test_data = preprocess_data("UCSDped1_test.npy")
        results = detect_anomalies(model, test_data, train_sequences)
        
        # Visualisation
        visualize_results(history, results, test_data)
        
        print("\n=== RÉSULTATS ===")
        print(f"Anomalies détectées: {results['anomaly_percentage']:.2f}%")
        print(f"Seuil utilisé: {results['threshold']:.4f}")
        print("Résultats sauvegardés dans le dossier 'results'")
        
    except Exception as e:
        print(f"\nErreur: {str(e)}")

if __name__ == "__main__":
    main()

1. Construction du modèle...


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 5, 64, 64, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_3 (ConvLSTM2D)      │ (None, 5, 64, 64, 16)  │         9,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 5, 64, 64, 16)  │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 5, 64, 64, 8)   │         1,160 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 5, 64, 64, 1)   │            73 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,153 (43.57 KB)

 Trainable params: 11,121 (43.44 KB)

 Non-trainable params: 32 (128.00 B)


2. Chargement des données d'entraînement...
Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 92ms/step - loss: 0.2458 - val_loss: 0.2437
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.2347 - val_loss: 0.2376
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.2147 - val_loss: 0.2289
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - loss: 0.1840 - val_loss: 0.2172
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - loss: 0.1495 - val_loss: 0.2050
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - loss: 0.1185 - val_loss: 0.1938
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step - loss: 0.0891 - val_loss: 0.1828
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - loss: 0.0624 - val_loss: 0.1709
Epoch 9/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - loss: 0.0420 - val_loss: 0.1591
Epoch 10/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0285 - val_loss: 0.1481

3. Détection des anomalies...
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 21m